In [ ]:
! pip install kaggle

In [2]:
! mkdir ~/.kaggle

In [3]:
! cp kaggle.json ~/.kaggle/

In [4]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets download arunrk7/surface-crack-detection


In [ ]:
!unzip surface-crack-detection.zip

In [44]:
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from torch.optim import Adam
import torch
import torch.nn as nn
import torch.optim as optim
from PIL import Image
import random
import os

In [8]:
image_list = []
for image in os.listdir('Negative'):
  root_path = os.path.join('Negative' , image)
  image_list.append((root_path , 0))

for image in os.listdir('Positive'):
  root_path = os.path.join('Positive' , image)
  image_list.append((root_path , 1))


In [ ]:
for x,y in image_list :
  print(x)
  print(y)

In [9]:
class Custom_dataset(Dataset):
  def __init__(self , image_list , transform):
    self.image_list = image_list
    self.transform = transform
    self.images = []
    self.labels = []

    for dir , label in image_list:
      self.images.append(dir)
      self.labels.append(label)

  def __len__(self):
    return len(self.images)

  def __getitem__(self, index) :
    image = Image.open(self.images[index])
    label = self.labels[index]

    if self.transform:
      image = self.transform(image)

    return image, label

In [ ]:
random.shuffle(image_list)
train_size = len(image_list) * 0.8
test_size = len(image_list) * 0.2

print(test_size)

8000.0


In [20]:
transform = transforms.Compose([transforms.Resize((256, 256)), transforms.ToTensor()])
train_dataset = Custom_dataset(image_list[0:32000] , transform)
test_dataset = Custom_dataset(image_list[32000 : 40000] , transform)

train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=True)

In [ ]:
for batch_idx, (data, labels) in enumerate(train_dataloader):
    print(f"Batch {batch_idx}")
    print("Data:", data)
    print("Labels:", labels)
    # Optionally, break after the first few batches to avoid printing too much data
    if batch_idx == 1:  # Change this number based on how many batches you want to print
        break

In [37]:
class Classifier(nn.Module):
    def __init__(self, nc, output):
        super(Classifier, self).__init__()
        self.nc = nc
        self.output = output
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(32 * 32 * 128, 32)  # Adjust the size
        self.fc2 = nn.Linear(32, 2)  # Assuming 2 classes: cracked and uncracked
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = self.pool(self.relu(self.conv3(x)))
        x = x.view(-1, 32 * 32 * 128)  # Adjust the size
        x = self.dropout(self.relu(self.fc1(x)))
        x = self.fc2(x)
        return x


In [38]:
input = torch.randn(32, 3 , 256, 256)
model = Classifier(3,2)
output = model(input)
output.shape


torch.Size([32, 2])

In [36]:
pool = nn.MaxPool2d(2, 2)
c1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
c2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
c3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)

o1 = pool(c1(input))
o2 = pool(c2(o1))
o3 = pool(c3(o2))

o4 = o3.view(-1, 32 * 32 * 128)

l1 = nn.Linear(32 * 32 * 128, 32)
o5 = l1(o4)

l2 = nn.Linear(32 , 2)
o6 = l2(o5)

print(o6.shape)

torch.Size([32, 2])


In [40]:
model = Classifier(3,2)
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Define the cross-entropy loss
criterion = nn.CrossEntropyLoss()

# Assuming you're using GPU (CUDA)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for iter in range(5):
    correct = 0.0
    total = 0.0
    running_loss = 0.0

    for batch_idx, (data, labels) in enumerate(train_dataloader):
        # Move data and labels to the GPU
        data, labels = data.to(device), labels.to(device)

        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(data)

        # Calculate the loss
        loss = criterion(outputs, labels)

        # Backpropagation
        loss.backward()

        # Update weights
        optimizer.step()

        # Track accuracy
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        # Accumulate loss
        running_loss += loss.item()

    # Print statistics for this iteration
    print(f'Iteration {iter + 1}:')
    print(f'Loss: {running_loss / len(train_dataloader):.4f}')
    print(f'Accuracy: {(correct / total) * 100:.2f}%')

Iteration 1:
Loss: 0.4890
Accuracy: 75.06%
Iteration 2:
Loss: 0.1247
Accuracy: 96.20%
Iteration 3:
Loss: 0.0818
Accuracy: 97.69%
Iteration 4:
Loss: 0.0668
Accuracy: 98.14%


In [46]:
criterion = nn.CrossEntropyLoss()  # Use the appropriate loss function

model.eval()  # Set the model to evaluation mode
test_loss = 0.0  # Initialize the test loss
correct = 0  # Initialize the number of correct predictions
total = 0  # Initialize the total number of samples

with torch.no_grad():  # Disable gradient computation for inference
    for data, labels in test_dataloader:
        data = data.to(device)  # Move data to the GPU if available
        labels = labels.to(device)  # Move labels to the GPU if available

        outputs = model(data)  # Get model predictions
        loss = criterion(outputs, labels)  # Calculate the loss

        test_loss += loss.item()  # Accumulate the loss
        _, predicted = torch.max(outputs, 1)  # Get the class with the highest probability
        total += labels.size(0)  # Increment the total number of samples
        correct += (predicted == labels).sum().item()  # Count correct predictions

# Calculate the average test loss
average_loss = test_loss / len(test_dataloader)
accuracy = 100.0 * correct / total

print(f'Test Loss: {average_loss:.4f}')
print(f'Test Accuracy: {accuracy:.2f}%')

Test Loss: 0.0895
Test Accuracy: 97.83%
